In [1]:
import os
import re
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import warnings
warnings.filterwarnings('ignore')

### feature selection: choosing the mss that will be analyzed, based on a filter of Wright's decoder +
### Wright's table of contents

In [2]:
new_master = pd.read_excel('trial master I.xlsx')
new_master

,Unnamed: 0,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,...,locusGrp,supplied,filiation,ptr,name,origPlace,orgName,finalrubric,desc,from folio
0,0,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1a1,False,107b,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ\n \...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3b
1,1,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition1,False,107b,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107b
2,2,1.xml,"Add. 12,133, foll. 1–108",8,IX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108a
3,3,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition2,True,108a,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108a
4,4,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition3,False,108a,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,22122,985.xml,"Add. 17,224, fol. 75",13,CCXLI,p14a2,False,NaN,A fragment of an Index of Lessons\n,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75b
22123,22123,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a1,False,NaN,A portion of the Index of the discourses conta...,ܩܘܕܩܣ ܕܡ̈ܐܡܪܐ ܘܕܬܘܪ̈ܓܡܐ ܕܒܟܬܒܐ ܗ[ܢܐ] ܡ̈ܛܟܣܝܢ. ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76a
22124,22124,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a2,False,NaN,"A note, of which the only interesting portion ...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76b
22125,22125,985.xml,"Add. 17,224, fol. 77",13,MXXX,p16a1,False,NaN,"A note stating that the volume, to which it be...",ܟܬܒܗ ܕܝܢ ܠܟܬܒܐ ܗܢܐ ܡܣܥܘܕ ܡܚܝ̣ܠܐ ܕܒܫܡܐ ܕܝܪܝܐ ܒܪ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77a


In [3]:
columns = ['title', 'note', 'author', 'rubric', 'from folio','persName', 'quote','finalRubric']

In [4]:
for col in columns:
    new_master[col] = new_master[col].str.replace('\n','')
    new_master[col] = new_master[col].str.replace('\\n', '')
    new_master[col] = new_master[col].str.replace('\\', '')
    new_master[col] = new_master[col].str.replace('[','')
    new_master[col] = new_master[col].str.replace(']', '')
    new_master[col] = new_master[col].str.replace('"', '')
    new_master[col] = new_master[col].str.replace(' n ', '')
    new_master[col] = new_master[col].str.replace("\'n ", '')
    new_master[col] = new_master[col].str.replace('  ', ' ')
    new_master[col] = new_master[col].str.replace('  ', ' ')
    
#new_master['persName'] = new_master['persName'].str.replace(':', '')
    
new_master['from folio'] = new_master['from folio'].str.replace(' ', '')
new_master.drop(columns=['Unnamed: 0'],inplace=True)
new_master


,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,finalRubric,...,locusGrp,supplied,filiation,ptr,name,origPlace,orgName,finalrubric,desc,from folio
0,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1a1,False,107b,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ,ܫܠܡ ܡܦܩܢܐ̇ ܣ̣ܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ;,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3b
1,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition1,False,107b,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107b
2,1.xml,"Add. 12,133, foll. 1–108",8,IX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108a
3,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition2,True,108a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108a
4,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition3,False,108a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,985.xml,"Add. 17,224, fol. 75",13,CCXLI,p14a2,False,NaN,A fragment of an Index of Lessons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75b
22123,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a1,False,NaN,A portion of the Index of the discourses conta...,ܩܘܕܩܣ ܕܡ̈ܐܡܪܐ ܘܕܬܘܪ̈ܓܡܐ ܕܒܟܬܒܐ ܗܢܐ ܡ̈ܛܟܣܝܢ.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76a
22124,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a2,False,NaN,"A note, of which the only interesting portion ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76b
22125,985.xml,"Add. 17,224, fol. 77",13,MXXX,p16a1,False,NaN,"A note stating that the volume, to which it be...",ܟܬܒܗ ܕܝܢ ܠܟܬܒܐ ܗܢܐ ܡܣܥܘܕ ܡܚܝ̣ܠܐ ܕܒܫܡܐ ܕܝܪܝܐ ܒܪ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77a


In [5]:
# the conversion of 'from folio' to numeric values
new_master['from folio ab'] = new_master['from folio']
new_master['from folio'] = new_master['from folio'].str.replace(r'[ab]', '')
new_master['from folio ab'] = new_master['from folio ab'].str.replace(r'\d', '')
new_master['from folio ab'] = new_master['from folio ab'].str.replace(r'a', '0')
new_master['from folio ab'] = new_master['from folio ab'].str.replace(r'b', '0.5')

new_master

,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,finalRubric,...,supplied,filiation,ptr,name,origPlace,orgName,finalrubric,desc,from folio,from folio ab
0,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1a1,False,107b,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ,ܫܠܡ ܡܦܩܢܐ̇ ܣ̣ܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ;,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0.5
1,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition1,False,107b,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107,0.5
2,1.xml,"Add. 12,133, foll. 1–108",8,IX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0
3,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition2,True,108a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0
4,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition3,False,108a,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,985.xml,"Add. 17,224, fol. 75",13,CCXLI,p14a2,False,NaN,A fragment of an Index of Lessons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,0.5
22123,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a1,False,NaN,A portion of the Index of the discourses conta...,ܩܘܕܩܣ ܕܡ̈ܐܡܪܐ ܘܕܬܘܪ̈ܓܡܐ ܕܒܟܬܒܐ ܗܢܐ ܡ̈ܛܟܣܝܢ.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,0
22124,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a2,False,NaN,"A note, of which the only interesting portion ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,0.5
22125,985.xml,"Add. 17,224, fol. 77",13,MXXX,p16a1,False,NaN,"A note stating that the volume, to which it be...",ܟܬܒܗ ܕܝܢ ܠܟܬܒܐ ܗܢܐ ܡܣܥܘܕ ܡܚܝ̣ܠܐ ܕܒܫܡܐ ܕܝܪܝܐ ܒܪ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,0


In [6]:
# the conversion of 'to folio' to numeric values
new_master['to folio ab'] = new_master['to folio']
new_master['to folio'] = new_master['to folio'].str.replace(r'[ab]', '')
new_master['to folio ab'] = new_master['to folio ab'].str.replace(r'\d', '')
new_master['to folio ab'] = new_master['to folio ab'].str.replace(r'a', '0')
new_master['to folio ab'] = new_master['to folio ab'].str.replace(r'b', '0.5')
new_master

,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,finalRubric,...,filiation,ptr,name,origPlace,orgName,finalrubric,desc,from folio,from folio ab,to folio ab
0,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1a1,False,107,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ,ܫܠܡ ܡܦܩܢܐ̇ ܣ̣ܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ;,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,0.5,0.5
1,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition1,False,107,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107,0.5,0.5
2,1.xml,"Add. 12,133, foll. 1–108",8,IX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0,NaN
3,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition2,True,108,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0,0
4,1.xml,"Add. 12,133, foll. 1–108",8,IX,p1addition3,False,108,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22122,985.xml,"Add. 17,224, fol. 75",13,CCXLI,p14a2,False,NaN,A fragment of an Index of Lessons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75,0.5,NaN
22123,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a1,False,NaN,A portion of the Index of the discourses conta...,ܩܘܕܩܣ ܕܡ̈ܐܡܪܐ ܘܕܬܘܪ̈ܓܡܐ ܕܒܟܬܒܐ ܗܢܐ ܡ̈ܛܟܣܝܢ.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,0,NaN
22124,985.xml,"Add. 17,224, fol. 76",12,MXXVI,p15a2,False,NaN,"A note, of which the only interesting portion ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76,0.5,NaN
22125,985.xml,"Add. 17,224, fol. 77",13,MXXX,p16a1,False,NaN,"A note stating that the volume, to which it be...",ܟܬܒܗ ܕܝܢ ܠܟܬܒܐ ܗܢܐ ܡܣܥܘܕ ܡܚܝ̣ܠܐ ܕܒܫܡܐ ܕܝܪܝܐ ܒܪ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77,0,NaN


In [8]:
new_master.to_excel('master I.xlsx',index=False)
#new_master = pd.read_excel('master I.xlsx')

In [14]:
len(new_master)

21734

In [12]:
new_master.shelfmark.describe()

count           21491
unique           1174
top       Add. 12,155
freq             1297
Name: shelfmark, dtype: object

In [9]:
#now we have a master_df that contains a list of all msItems that are in the BL catalogue.
# overview of all msItems - get descriptives, total number, maybe also per genre, and for centuries.

In [13]:
new_master.drop_duplicates(ignore_index=True, inplace=True)
len(new_master)

21734

In [ ]:
new_master.reset_index(drop = True, inplace=True)


In [10]:
#make ab columns numeric, so sum folio can be produced
new_master['from folio'] = new_master['from folio'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
new_master['from folio ab'] = new_master['from folio ab'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
new_master['to folio'] = new_master['to folio'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
new_master['to folio ab'] = new_master['to folio ab'].apply(lambda x: pd.to_numeric(x, errors='coerce'))
new_master['date'] = new_master['date'].apply(lambda x: pd.to_numeric(x, errors='coerce'))


In [11]:
# new_master['from folio'].dropna(inplace=True)
# new_master['date'].dropna(inplace=True)
# new_master['shelfmark'].dropna(inplace=True)
new_master = new_master.drop(new_master[new_master['date']>20].index)
new_master = new_master.dropna(subset=['date'])
new_master = new_master.dropna(subset=['shelfmark'])
new_master = new_master.dropna(subset=['from folio'])
new_master.reset_index(drop = True, inplace=True)
new_master

,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,finalRubric,...,filiation,ptr,name,origPlace,orgName,finalrubric,desc,from folio,from folio ab,to folio ab
0,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1a1,False,107.0,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ,ܫܠܡ ܡܦܩܢܐ̇ ܣ̣ܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ;,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,0.5,0.5
1,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition1,False,107.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,0.5,0.5
2,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,0.0,NaN
3,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition2,True,108.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,0.0,0.0
4,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition3,False,108.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19911,985.xml,"Add. 17,224, fol. 75",13.0,CCXLI,p14a2,False,NaN,A fragment of an Index of Lessons,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.0,0.5,NaN
19912,985.xml,"Add. 17,224, fol. 76",12.0,MXXVI,p15a1,False,NaN,A portion of the Index of the discourses conta...,ܩܘܕܩܣ ܕܡ̈ܐܡܪܐ ܘܕܬܘܪ̈ܓܡܐ ܕܒܟܬܒܐ ܗܢܐ ܡ̈ܛܟܣܝܢ.,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,0.0,NaN
19913,985.xml,"Add. 17,224, fol. 76",12.0,MXXVI,p15a2,False,NaN,"A note, of which the only interesting portion ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.0,0.5,NaN
19914,985.xml,"Add. 17,224, fol. 77",13.0,MXXX,p16a1,False,NaN,"A note stating that the volume, to which it be...",ܟܬܒܗ ܕܝܢ ܠܟܬܒܐ ܗܢܐ ܡܣܥܘܕ ܡܚܝ̣ܠܐ ܕܒܫܡܐ ܕܝܪܝܐ ܒܪ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,77.0,0.0,NaN


In [12]:
new_master['to folio'].isna().sum()

12666

In [13]:
#first, filter out undated items
trial_master = new_master.copy()

len(trial_master)

19916

In [14]:
trial_master['to folio'].isna().sum()

12666

In [15]:
trial_master['to folio'].count()

7250

In [16]:
trial_master.sort_values(by=['filename','from folio', 'from folio ab', 'to folio', 'to folio ab'], inplace=True, ignore_index=True)
for i in range(len(trial_master)):
    if not np.isnan(trial_master['to folio'][i]):
        if np.isnan(trial_master['from folio ab'][i]):
            trial_master['from folio ab'][i] = 0
        if i == len(trial_master)-1:
            trial_master['to folio'][i] = trial_master['from folio'][i]
            trial_master['to folio ab'][i] = trial_master['from folio ab'][i]
        elif trial_master['filename'][i]==trial_master['filename'][i+1]:
            trial_master['to folio'][i] = trial_master['from folio'][i+1]
            if not np.isnan(trial_master['from folio ab'][i+1]):
                trial_master['to folio ab'][i] = trial_master['from folio ab'][i+1]
        else:
            trial_master['to folio'][i] = trial_master['from folio'][i]
            if not np.isnan(trial_master['from folio ab'][i]):
                trial_master['to folio ab'][i] = trial_master['from folio ab'][i]
                


In [17]:
trial_master['to folio ab'].isna().sum()

12958

In [26]:
#need to fix length. start from begining of this notebook.
trial_master['length'] = None


trial_master['from folio sum'] = trial_master.apply(lambda x: (float(x['from folio'])+float(x['from folio ab'])), axis=1)
trial_master['to folio sum'] = trial_master.apply(lambda x: (float(x['to folio'])+float(x['to folio ab'])), axis=1)

for i in range(len(trial_master)):
    x = trial_master['to folio sum'][i]-trial_master['from folio sum'][i]#+trial_master['from folio ab'][i])
    trial_master['length'][i] = pd.to_numeric(x, errors='coerce')


trial_master          

,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,finalRubric,...,finalrubric,desc,from folio,from folio ab,to folio ab,length,from folio sum,to folio sum,Wright Section,Wright Sub-Section
0,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition4,False,2.0,NaN,NaN,NaN,...,NaN,NaN,2.0,0.0,0.0,0.0,2.0,2.0,Biblical Manuscripts,Old Testament
1,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition5,False,3.0,NaN,NaN,NaN,...,NaN,NaN,2.0,0.0,0.5,1.5,2.0,3.5,Biblical Manuscripts,Old Testament
2,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1a1,False,107.0,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ,ܫܠܡ ܡܦܩܢܐ̇ ܣ̣ܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ;,...,NaN,NaN,3.0,0.5,0.5,104.0,3.5,107.5,Biblical Manuscripts,Old Testament
3,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition1,False,108.0,NaN,NaN,NaN,...,NaN,NaN,107.0,0.5,0.0,0.5,107.5,108.0,Biblical Manuscripts,Old Testament
4,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition2,True,108.0,NaN,NaN,NaN,...,NaN,NaN,108.0,0.0,0.0,0.0,108.0,108.0,Biblical Manuscripts,Old Testament
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19911,985.xml,"Add. 17,224, fol. 75",13.0,CCXLI,p14a2,False,NaN,A fragment of an Index of Lessons,NaN,NaN,...,NaN,NaN,75.0,0.5,NaN,NaN,75.5,NaN,Service Books,Lectionaries
19912,985.xml,"Add. 17,224, fol. 76",12.0,MXXVI,p15a1,False,NaN,A portion of the Index of the discourses conta...,ܩܘܕܩܣ ܕܡ̈ܐܡܪܐ ܘܕܬܘܪ̈ܓܡܐ ܕܒܟܬܒܐ ܗܢܐ ܡ̈ܛܟܣܝܢ.,NaN,...,NaN,NaN,76.0,0.0,NaN,NaN,76.0,NaN,Scientific Literature,Natural History
19913,985.xml,"Add. 17,224, fol. 76",12.0,MXXVI,p15a2,False,NaN,"A note, of which the only interesting portion ...",NaN,NaN,...,NaN,NaN,76.0,0.5,NaN,NaN,76.5,NaN,Scientific Literature,Natural History
19914,985.xml,"Add. 17,224, fol. 77",13.0,MXXX,p16a1,False,NaN,"A note stating that the volume, to which it be...",ܟܬܒܗ ܕܝܢ ܠܟܬܒܐ ܗܢܐ ܡܣܥܘܕ ܡܚܝ̣ܠܐ ܕܒܫܡܐ ܕܝܪܝܐ ܒܪ...,NaN,...,NaN,NaN,77.0,0.0,NaN,NaN,77.0,NaN,Scientific Literature,Natural History


In [27]:
trial_master['length'].count()

6957

In [20]:
df = pd.read_excel("Decoder+TOC.xlsx")
df.set_index("Wright Numeration", inplace = True)


In [21]:
#adding a column from the base df
trial_master['Wright Section'] = None
for i in range(len(trial_master)):
    trial_master['Wright Section'][i] = df['Wright Section'].loc[trial_master['roman'][i]]

trial_master['Wright Sub-Section'] = None
for i in range(len(trial_master)):
    trial_master['Wright Sub-Section'][i] = df['Wright Sub-Section'].loc[trial_master['roman'][i]]
    
trial_master.head()

,filename,shelfmark,date,roman,ID,overtitle,to folio,title,rubric,finalRubric,...,finalrubric,desc,from folio,from folio ab,to folio ab,length,from folio sum,to folio sum,Wright Section,Wright Sub-Section
0,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition4,False,2.0,NaN,NaN,NaN,...,NaN,NaN,2.0,0.0,0.0,0.0,2.0,2.0,Biblical Manuscripts,Old Testament
1,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition5,False,3.0,NaN,NaN,NaN,...,NaN,NaN,2.0,0.0,0.5,1.5,2.0,3.5,Biblical Manuscripts,Old Testament
2,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1a1,False,107.0,"The book of Exodus, according to the Peshitta ...",ܡܦܩܢܐ̇ ܣܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ,ܫܠܡ ܡܦܩܢܐ̇ ܣ̣ܦܪܐ ܕܬܪ̈ܝܼܢ ܕܐܘܪܝܬܐ;,...,NaN,NaN,3.0,0.5,0.5,104.0,3.5,107.5,Biblical Manuscripts,Old Testament
3,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition1,False,108.0,NaN,NaN,NaN,...,NaN,NaN,107.0,0.5,0.0,0.5,107.5,108.0,Biblical Manuscripts,Old Testament
4,1.xml,"Add. 12,133, foll. 1–108",8.0,IX,p1addition2,True,108.0,NaN,NaN,NaN,...,NaN,NaN,108.0,0.0,0.0,0.0,108.0,108.0,Biblical Manuscripts,Old Testament


In [28]:
trial_master.isna().sum()#['shelfmark'].count()

filename                  0
shelfmark                 0
date                      0
roman                     0
ID                     3787
overtitle              3796
to folio              12666
title                  4720
rubric                11083
finalRubric           19365
colophon              19620
note                  15044
label                 19675
placeName             19877
listPerson            19903
persName              16963
quote                 18071
bibl                  19656
ref                   19701
foreign               19710
editor                19898
author                14053
incipit               17934
persName+             19764
explicit              19820
listBibl              19214
sic                   19912
table                 19915
material              19915
choice                19902
origDate              19912
locusGrp              19916
supplied              19914
filiation             19915
ptr                   19916
name                

In [29]:
trial_master.to_excel('item master with subsection new.xlsx')